### Imports and database initialization

In [ ]:
from sqlalchemy import select, func
from gorillatracker.ssl_pipeline.models import Tracking, Video, Camera
from sqlalchemy.orm import sessionmaker
from gorillatracker.ssl_pipeline.dataset import GorillaDatasetKISZ

from tabulate import tabulate

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

import numpy as np

engine = GorillaDatasetKISZ().engine
session_cls = sessionmaker(bind=engine)

### Number of frames per camera (sorted descending)

In [ ]:
stmt = (
    select(Video.camera_id, func.sum(Video.frames).label("count"))
    .group_by(Video.camera_id)
    )

with session_cls() as session:
    result = session.execute(stmt)
    sorted_all = sorted(result.fetchall(), key=lambda x: x[1], reverse=True)
    print(tabulate(sorted_all, headers=result.keys(), tablefmt="psql"))

### Number of Tracking-IDs per Video (not really meaningful with 250000 videos)

In [ ]:
stmt = (
    select(Tracking.video_id, func.count(Tracking.tracking_id).label("count"))
    .group_by(Tracking.video_id)
    )

with session_cls() as session:
    result = session.execute(stmt)
    sorted_all = sorted(result.fetchall(), key=lambda x: x[1], reverse=True)
    
list1, list2 = zip(*sorted_all)
plt.bar(list1, list2)
plt.xlabel("Video")
plt.ylabel("Number of ID's")
plt.ylim(0, 40)
plt.tight_layout()
plt.show()

### Number of Tracking-IDs per camera

In [ ]:
from ipywidgets import IntProgress

stmt = (
    select(Video.camera_id, func.sum(Tracking.tracking_id).label("count"))
    .join(Tracking, Video.video_id == Tracking.video_id)
    .group_by(Video.camera_id)
    )

with session_cls() as session:
    result = session.execute(stmt)
    all = sorted(result.fetchall(), key=lambda x: x[0])
    print(tabulate(all, headers=result.keys(), tablefmt="psql"))

stmt = (
    select(Camera.camera_id, Camera.longitude, Camera.latitude)
    )

with session_cls() as session:
    result = session.execute(stmt)
    all2 = sorted(result.fetchall(), key=lambda x: x[0])
    print(tabulate(all2, headers=result.keys(), tablefmt="psql"))

mapping = {x : (y,z) for x, y, z in all2}

f = IntProgress(min=0, max=len(all)) # instantiate the bar
display(f)

points = []
for (camera_id2, count) in all:
    f.value += 1
    (long, lat) = mapping[camera_id2]
    if long is None or lat is None:
        continue
    for i in range(count//10000):
        points.append((long + np.random.normal(0, 0.002), lat + np.random.normal(0, 0.002)))

x, y = zip(*points)

plt.hist2d(x,y, bins=1000, cmap="twilight")
plt.show()

### Average number of Tracking-IDs in videos per camera

In [ ]:
stmt = (
    select(Video.camera_id, func.avg(Tracking.tracking_id).label("count"))
    .join(Tracking, Video.video_id == Tracking.video_id)
    .group_by(Video.camera_id)
    )

with session_cls() as session:
    result = session.execute(stmt)
    all = sorted(result.fetchall(), key=lambda x: x[0])
    print(tabulate(all, headers=result.keys(), tablefmt="psql"))
    
list1, list2 = zip(*all)
plt.bar(list1, list2)
plt.xlabel("Camera")
plt.ylabel("Avg. number of ID's")
plt.tight_layout()
plt.show()

### Number of Videos per Date

In [ ]:
with session_cls() as session:
    result = session.execute(select(Video.start_time))
    all = result.fetchall()
    print(tabulate(all, headers=result.keys(), tablefmt="psql"))
    
all_dates = [mdates.date2num(date[0].date()) for date in all if date[0] is not None and date[0].date().year > 2000]

fig, ax = plt.subplots()
ax.hist(all_dates, bins=100)
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
plt.xlabel("Date")
plt.ylabel("Number of Videos")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Length of Videos

In [ ]:
stmt = (
    select((Video.frames / Video.fps).label("duration"), func.count(Video.video_id).label("count"))
    .group_by("duration")
    )

with session_cls() as session:
    result = session.execute(stmt)
    all = result.fetchall()

list1, list2 = zip(*all)
plt.bar(list1, list2)
plt.xlabel("Length (seconds)")
plt.ylabel("Number of Videos")
plt.yscale("log")
plt.tight_layout()
plt.show()

### Distribution of Video Recording Start Times

In [ ]:
from datetime import datetime

stmt = (
    select(Video.start_time)
)

with session_cls() as session:
    result = session.execute(stmt)
    all = result.fetchall()
    all_times = [mdates.date2num(datetime.combine(datetime.today(), t[0].time())) for t in all if t[0] is not None]
    
fig, ax = plt.subplots()
ax.hist(all_times, bins=24 * 4, rwidth=0.8)
ax.xaxis.set_major_formatter(DateFormatter("%H:%M"))
plt.xlabel("Time of Day Start Time")
plt.ylabel("Number of Videos")
plt.title("Distribution of Video Recording Times")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()